In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3893/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_10_49_2,0.999962,0.677023,0.697356,0.999921,0.000014,0.191733,0.212008,0.000023,0.001383,0.003781,1.000007,0.003810,1608.311558,3882.266378,"Hidden Size=[36, 18], regularizer=0.02, learni..."
1,model_10_49_1,0.999962,0.677056,0.697351,0.999923,0.000014,0.191713,0.212012,0.000023,0.001392,0.003793,1.000007,0.003823,1608.297965,3882.252785,"Hidden Size=[36, 18], regularizer=0.02, learni..."
2,model_10_49_0,0.999961,0.677197,0.697503,0.999924,0.000014,0.191630,0.211906,0.000022,0.001394,0.003808,1.000007,0.003837,1608.282814,3882.237633,"Hidden Size=[36, 18], regularizer=0.02, learni..."
3,model_10_48_9,0.999961,0.677256,0.697534,0.999924,0.000015,0.191595,0.211883,0.000022,0.001401,0.003822,1.000008,0.003852,1608.267838,3882.222657,"Hidden Size=[36, 18], regularizer=0.02, learni..."
4,model_10_48_8,0.999961,0.677289,0.697547,0.999924,0.000015,0.191575,0.211875,0.000022,0.001411,0.003847,1.000008,0.003877,1608.241578,3882.196398,"Hidden Size=[36, 18], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4990,model_5_0_0,-0.084002,-0.267533,-0.178359,-0.392476,0.408070,0.752462,0.825465,0.526018,0.315409,0.638804,9.739766,0.643775,291.792631,707.585126,"Hidden Size=[36], regularizer=0.02, learning_r..."
4991,model_1_0_2,-0.098773,-0.107310,-0.312892,-0.175542,0.413631,0.657347,0.919708,0.374647,0.333784,0.643141,-0.265551,0.648147,35.765562,84.513648,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4992,model_4_0_0,-0.105508,0.153433,0.571284,-1.242525,0.416166,0.502559,0.300324,0.504811,0.261225,0.645110,-3.456580,0.650130,195.753340,473.904182,"Hidden Size=[24], regularizer=0.02, learning_r..."
4993,model_1_0_1,-0.119926,-0.121994,-0.329264,-0.200379,0.421594,0.666064,0.931177,0.382563,0.336901,0.649303,-0.289915,0.654356,35.727425,84.475510,"Hidden Size=[4], regularizer=0.02, learning_ra..."
